In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [3]:
!pip install keras keras-rl2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.9 MB/s eta 0:00:00


In [4]:
!kaggle datasets download -d niharika41298/gym-exercise-data

Dataset URL: https://www.kaggle.com/datasets/niharika41298/gym-exercise-data
License(s): CC0-1.0
  0% 0.00/120k [00:00<?, ?B/s]
100% 120k/120k [00:00<00:00, 55.9MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gym-exercise-data.zip', 'r')
zip_ref.extractall('/content/alldata')
zip_ref.close()

In [7]:
from gym import Env
from gym import spaces
from gym.spaces import Discrete, Box
import numpy as np
import random
import pandas as pd

In [8]:
data = pd.read_csv('/content/alldata/megaGymDataset.csv')
new_df = data.drop(columns=["Rating","RatingDesc"])


In [9]:
new_df.rename(columns={'Unnamed: 0': 'S.no', 'Unnamed: 1': 'NewName2'}, inplace=True)

In [10]:
new_df.reset_index(drop=True,inplace=True)

In [11]:
new_df.to_csv('new_data.csv')

In [16]:
import numpy as np
import pandas as pd
from gym import Env, spaces

class fitness_model:
    def __init__(self, difficulty=None, purpose=None):
        self.action_space = spaces.Discrete(2918)
        self.observation_space = spaces.MultiDiscrete([2918, 2918, 2918, 2918, 2918])
        self.state = np.random.randint(0, 2918, size=5)
        self.reward = 0
        self.difficulty = difficulty
        self.purpose = purpose
        self.new_df = pd.read_csv("new_data.csv")
        self.step_count = 0

    def selector(self):
        arr = self.state.tolist()

        for k in range(4):
            temp_row = self.new_df.iloc[self.state[k]]
            level_match = temp_row["Level"] == self.difficulty
            type_match = temp_row["Type"] == self.purpose

            if not level_match or not type_match:
                break
            elif len(temp_row["BodyPart"]) == len(np.unique(temp_row["BodyPart"])):
                break

        return k


    def step(self, action):
        self.reward = 0

        exercise_to_replace = self.selector()
        new_exercise = action

        if new_exercise not in self.state:
            self.state[exercise_to_replace] = new_exercise



        if len(self.state) == len(np.unique(self.state)):
            diff = 2
        else:
            diff = -1

        array = self.state.tolist()
        temp = self.new_df.iloc[array]


        if temp.shape[0] > 0:
            if len(temp["BodyPart"]) == len(np.unique(temp["BodyPart"])):
                diverse = 1
            else:
                diverse = -1
        else:
            diverse = -1


        pref_level = 0
        for k in self.state:
            if self.new_df.loc[k, "Level"] == self.difficulty and self.difficulty is not None:
                pref_level += 1
                break

        pref_purpose = 0
        for k in self.state:
            if self.new_df.loc[k, "Type"] != self.purpose and self.purpose is not None:
                pref_purpose += 1


        self.reward = (3 * diverse) +  pref_purpose + pref_level + diff

        if self.reward >= 10  :
            done = True
            self.step_count = 0
        else:
            done = False

        info = {}
        self.step_count+=1
        return self.state, self.reward, done, info

    def render(self):
        print(self.state)

    def filter_exercises(self):

            if self.difficulty and self.purpose:
                return self.new_df[(self.new_df["Level"] == self.difficulty) & (self.new_df["Type"] != self.purpose)]
            return self.new_df

    def reset(self):

        self.state = np.random.randint(0, 2918, size=5)

        return self.state





In [ ]:
import tensorflow as tf
import numpy as np
import gym

# Environment setup
env = fitness_model("Beginner","Strength")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Hyperparameters
gamma = 0.99  # Discount factor
lr_actor = 0.001  # Actor learning rate
lr_critic = 0.001  # Critic learning rate
clip_ratio = 0.2  # PPO clip ratio
epochs = 10  # Number of optimization epochs
batch_size = 64  # Batch size for optimization

# Actor and Critic networks
class ActorCritic(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(ActorCritic, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.policy_logits = tf.keras.layers.Dense(action_size)
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.value = tf.keras.layers.Dense(1)

    def call(self, state):
        x = self.dense1(state)
        logits = self.policy_logits(x)
        value = self.value(x)
        return logits, value

# PPO algorithm
def ppo_loss(old_logits, old_values, advantages, states, actions, returns):
    def compute_loss(logits, values, actions, returns):
        actions_onehot = tf.one_hot(actions, action_size, dtype=tf.float32)
        policy = tf.nn.softmax(logits)
        action_probs = tf.reduce_sum(actions_onehot * policy, axis=1)
        old_policy = tf.nn.softmax(old_logits)
        old_action_probs = tf.reduce_sum(actions_onehot * old_policy, axis=1)

        # Policy loss
        ratio = tf.exp(tf.math.log(action_probs + 1e-10) - tf.math.log(old_action_probs + 1e-10))
        clipped_ratio = tf.clip_by_value(ratio, 1 - clip_ratio, 1 + clip_ratio)
        policy_loss = -tf.reduce_mean(tf.minimum(ratio * advantages, clipped_ratio * advantages))

        # Value loss
        value_loss = tf.reduce_mean(tf.square(values - returns))

        # Entropy bonus (optional)
        entropy_bonus = tf.reduce_mean(policy * tf.math.log(policy + 1e-10))

        total_loss = policy_loss + 0.5 * value_loss - 0.01 * entropy_bonus  # Entropy regularization
        return total_loss

    def get_advantages(returns, values):
        advantages = returns - values
        return (advantages - tf.reduce_mean(advantages)) / (tf.math.reduce_std(advantages) + 1e-8)

    def train_step(states, actions, returns, old_logits, old_values):
        with tf.GradientTape() as tape:
            logits, values = model(states)
            loss = compute_loss(logits, values, actions, returns)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss

    advantages = get_advantages(returns, old_values)
    for _ in range(epochs):
        loss = train_step(states, actions, returns, old_logits, old_values)
    return loss


# Initialize actor-critic model and optimizer
model = ActorCritic(state_size, action_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_actor)

# Main training loop
max_episodes = 1000
max_steps_per_episode = 200

for episode in range(max_episodes):
    states, actions, rewards, values, returns = [], [], [], [], []
    state = env.reset()
    for step in range(max_steps_per_episode):
        state = tf.expand_dims(tf.convert_to_tensor(state), 0)
        logits, value = model(state)

        # Sample action from the policy distribution
        action = tf.random.categorical(logits, 1)[0, 0].numpy()
        next_state, reward, done, _ = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        values.append(value)

        state = next_state

        if done:
            returns_batch = []
            discounted_sum = 0
            for r in rewards[::-1]:
                discounted_sum = r + gamma * discounted_sum
                returns_batch.append(discounted_sum)
            returns_batch.reverse()

            states = tf.concat(states, axis=0)
            actions = np.array(actions, dtype=np.int32)
            values = tf.concat(values, axis=0)
            returns_batch = tf.convert_to_tensor(returns_batch)
            old_logits, _ = model(states)

            loss = ppo_loss(old_logits, values, returns_batch - np.array(values), states, actions, returns_batch)
            print(f"Episode: {episode + 1}, Loss: {loss.numpy()},State: {next_state}")

            break

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'actor_critic_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Episode: 2, Loss: 240.26754760742188,State: [2344  432 2838  453 2839]
Episode: 4, Loss: 2531.9443359375,State: [2745 1230  848 2159 2500]
Episode: 6, Loss: 3351.150390625,State: [2745 2124   96 2277 2275]
Episode: 8, Loss: 11035.8955078125,State: [ 990 2745 2046 1583 1754]
Episode: 10, Loss: 1764.2806396484375,State: [2745  253 1690 2129  550]
Episode: 12, Loss: 5919.92822265625,State: [2886 2313 1035 1647  385]
Episode: 14, Loss: 7634.45556640625,State: [2886 1747  165 2371 2836]
Episode: 16, Loss: 7666.92138671875,State: [2886 2686 2083  560  229]
Episode: 18, Loss: 14015.3427734375,State: [2745 1210 2263 1918  542]
Episode: 20, Loss: 6141.70703125,State: [2745 2306 1039 2868  206]
Episode: 22, Loss: 3569.700927734375,State: [2745  340 2388  895 2506]
Episode: 24, Loss: 1384.6776123046875,State: [2745 2002 2263 2316 2032]
Episode: 26, Loss: 7290.43603515625,State: [2692 2745 2374 2339  852]
Episode: 28, Loss: 21415.109375,State: [2745  836 1177  886 1912]
Episode: 30, Loss: 18062.54

In [19]:
final_df = pd.read_csv("new_data.csv")

In [27]:
# Function to test the trained agent
def test_agent(env, model, num_episodes=3, max_steps=300):
    for episode in range(num_episodes):
        state = env.reset()  # Reset the environment
        total_reward = 0  # Initialize total reward for the episode
        print(f"Testing Episode {episode + 1}:")

        for step in range(max_steps):
            state_tensor = tf.expand_dims(tf.convert_to_tensor(state), 0)  # Convert to tensor
            logits, _ = model(state_tensor)  # Get action logits

            # Sample action from the policy distribution
            action = tf.random.categorical(logits, 1)[0, 0].numpy()  # Sample an action
            next_state, reward, done, _ = env.step(action)  # Take action in the environment

            total_reward += reward  # Accumulate reward
            state = next_state  # Update state

            # Optional: Render the environment (if your env supports it)
            # env.render()

            if done:
                break  # End episode if done

        print(f"Total Reward for Episode {episode + 1}:{reward}, {final_df.iloc[env.state]}")

# Testing the agent
test_agent(env, model, num_episodes=1)  # Test the agent for 10 episodes


Testing Episode 1:
Total Reward for Episode 1:0,       Unnamed: 0  S.no                              Title  \
2745        2745  2745               PVC Shoulder Stretch   
56            56    56  HM Alternating Dumbbell Plank Row   
562          562   562           Holman Side Plank Crunch   
926          926   926                     Band chest fly   
87            87    87                 Lying Leg Pullover   

                                                   Desc        Type  \
2745                                                NaN  Stretching   
56                                                  NaN    Strength   
562                                                 NaN    Strength   
926   Similar to the cable chest fly, the band chest...    Strength   
87                                                  NaN    Strength   

        BodyPart  Equipment         Level  
2745   Shoulders  Body Only  Intermediate  
56    Abdominals   Dumbbell  Intermediate  
562   Abdominals  Body On